# OpenStreetMap example

In this example, we download a road network from OSM using the OSMNx package, and then process the result, resulting in a RouteE Compass network dataset.

## requirements

To download an open street maps dataset, we'll need some extra dependnecies which are included with the conda distribution of this pacakge: 

```console
conda create -n routee-compass -c conda-forge python=3.11 nrel.routee.compass 
```

In [21]:
import osmnx as ox
from nrel.routee.compass.io import generate_compass_dataset
from nrel.routee.compass import CompassApp
import json

## Building RouteE Compass Dataset

### Get OSM graph

First, we need to get an OSM graph that we can use to convert into the format RouteE Compass expects.

In this example we will load in a road network that covers Golden, Colorado as a small example, but this workflow will work with any osmnx graph (osmnx provides [many graph download operations](https://osmnx.readthedocs.io/en/stable/user-reference.html#module-osmnx.graph)).

In [2]:
g = ox.graph_from_place("Golden, Colorado, USA", network_type="drive")  

### Convert Graph to Compass Dataset

Now, we call the `generate_compass_dataset` function which will convert the osmnx graph into files that are compatible with RouteE Compass.

```{note}
In order to get the most accurate energy results from the routee-powertrain vehicle models, it's important to include road grade information since it plays a large factor in vehicle energy consumption (`add_grade=True`)
That being said, adding grade can be a big lift computationally. In our case, we pull digital elevation model (DEM) raster files from USGS and then use osmnx to append elevation and grade to the graph. If the graph is large, this can take a while to download and could take up a lot of disk space.
So, we recommend that you include grade information in your graph but want to be clear about the requirements for doing so.
```

In [5]:
generate_compass_dataset(g, output_directory="golden_co", add_grade=True)      

This will parse the OSM graph and write the RouteE Compass files into a new folder "golden_co/". If you take a look in this directory, you'll notice some `.toml` files like: `osm_default_energy.toml`. 
These are configurations for the compass application. Take a look [here](https://nrel.github.io/routee-compass/config.html) for more information about this file.

## Running

### Load Application

Now we can load the application from one of our config files. 
We'll pick `osm_default_energy.toml` for computing energy optimal routes.

In [6]:
app = CompassApp.from_config_file("golden_co/osm_default_energy.toml")  

uuid file: 100%|██████████| 746/746 [00:00<00:00, 2224495.00it/s]it/s]

###  Queries

With our application loaded we can start computing routes by passing queries to the app.
To demonstrate, we'll route between two locations in Golden, CO utilzing the grid search input plugin to run two separate searches. 

The energy cost coefficient indicates how much we should factor energy into our route search, 0.0 indicating that we should not factor in energy at all (shortest time route) and 1.0 indicating that we should only factor energy in (least energy route)

The `model_name` parameter specifies which energy model to use. This can anything that was included in the config as a `[[traveral.energy_models]]` section. We'll use the 2016 Toyota Camry.

In [13]:
query = [
    {
        "origin_name": "Government Center Station",
        "destination_name": "Cannonball Creek Brewery",
        "origin_x": -105.200146,
        "origin_y": 39.726570,
        "destination_x": -105.234964,
        "destination_y": 39.768477,
        "model_name": "2016_TOYOTA_Camry_4cyl_2WD",
        "state_variable_coefficients": {
            "distance": 0.0,
            "time": 1.0,
            "energy_liquid": 0.0,
        },
    },
    {
        "origin_name": "Government Center Station",
        "destination_name": "Cannonball Creek Brewery",
        "origin_x": -105.200146,
        "origin_y": 39.726570,
        "destination_x": -105.234964,
        "destination_y": 39.768477,
        "model_name": "2016_TOYOTA_Camry_4cyl_2WD",
        "state_variable_coefficients": {
            "distance": 0.0,
            "time": 0.0,
            "energy_liquid": 1.0,
        },

    }
]

Now, let's pass the query to the application.

```{note}
A query can be a single object, or, a list of objects.
If the input is a list of objects, the application will run these queries in parallel over the number of threads defined in the config file under the `paralellism` key (defaults to 2).
```

In [14]:
results = app.run(query)

search: 100%|██████████| 2/2 [00:00<00:00, 448.75it/s]2.20it/s]

## Analysis

The application returns the results as a list of python dictionaries.
Since we used the grid search to specify two separate searches, we should get two results back:

In [18]:
assert len(results), f"expected 2 results, found {len(results)}"
for r in results:
    error = r.get("error")
    if error is not None:
        print(f"request {r['request']['name']} had error: {error}")

### Traversal and Cost Summaries

Since we have the `traversal` output plugin activated by default, we can take a look at the summary for each result under the `traversal_summary` key.

In [53]:
def pretty_print(dict):
    print(json.dumps(dict, indent=4))
shortest_time_result = results[0]
least_energy_result = results[1]

Summary of route result for distance, time, and energy:

In [76]:

pretty_print(shortest_time_result["traversal_summary"])

{
    "distance": 4.23738036063421,
    "distance_unit": "miles",
    "energy_liquid": 1.7531906338670376,
    "time": 8.486183959367098,
    "time_unit": "minutes",
    "vehicle_info": {
        "energy_unit": "gallons_gasoline"
    }
}


The cost summary shows the costs per unit assigned to the trip, in dollars.

This is based on the user assumptions assigned in the configuration which can be overriden in the route request query.

In [78]:
pretty_print(shortest_time_result["cost_summary"]["cost"])

{
    "distance": 2.775484136215408,
    "energy_liquid": 5.469954777665158,
    "time": 2.8287506162795912,
    "total_cost": 11.074189530160158
}


The info section includes details for how these costs were calculated. 

The user can set different state variable coefficients in the query that are weighted against the vehicle state variable rates.

The algorithm will rely on the weighted costs while the cost summary will show the final costs without weight coefficients applied.

In [ ]:
shortest_time_result["cost_summary"]["info"]

Each response object contains this information. The least energy traversal and cost summary are below.

In [79]:
pretty_print(least_energy_result["traversal_summary"])
pretty_print(least_energy_result["cost_summary"]["cost"])

{
    "distance": 4.020870727313324,
    "distance_unit": "miles",
    "energy_liquid": 1.7423185323233328,
    "time": 8.587012115596758,
    "time_unit": "minutes",
    "vehicle_info": {
        "energy_unit": "gallons_gasoline"
    }
}
{
    "distance": 2.6336703263902272,
    "energy_liquid": 5.436033820848799,
    "time": 2.862360270564561,
    "total_cost": 10.932064417803588
}


What becomes interesting is if we can compare our choices. Here's a quick comparison of the two:

In [82]:
dist_diff = shortest_time_result["traversal_summary"]["distance"] - least_energy_result["traversal_summary"]["distance"]
time_diff = shortest_time_result["traversal_summary"]["time"] - least_energy_result["traversal_summary"]["time"]
enrg_diff = shortest_time_result["traversal_summary"]["energy_liquid"] - least_energy_result["traversal_summary"]["energy_liquid"]
cost_diff = shortest_time_result["cost_summary"]["cost"]["total_cost"] - least_energy_result["cost_summary"]["cost"]["total_cost"]
dist_unit = shortest_time_result["traversal_summary"]["distance_unit"]
time_unit = shortest_time_result["traversal_summary"]["time_unit"]
enrg_unit = shortest_time_result["traversal_summary"]["vehicle_info"]["energy_unit"]
print(f" - distance: {dist_diff:.2f} {dist_unit} further with energy-optimal")
print(f" - time: {dist_diff:.2f} {time_unit} longer with energy-optimal")
print(f" - energy: {enrg_diff:.2f} {enrg_unit} more with time-optimal")
print(f" - cost: ${cost_diff:.2f} more with energy-optimal")

 - distance: 0.22 miles further with energy-optimal
 - time: 0.22 minutes longer with energy-optimal
 - energy: 0.01 gallons_gasoline more with time-optimal
 - cost: $0.14 more with energy-optimal


In addition to the summary, the result also contains much more information.
Here's a list of all the different sections that get returned:

In [71]:
def print_keys(d, indent=0):
    for k in sorted(d.keys()):
        print(f"{' '*indent} - {k}")
        if isinstance(d[k], dict):
            print_keys(d[k], indent+2)
print_keys(least_energy_result)

 - algorithm_runtime
 - basic_summary_runtime
 - cost_summary
   - cost
     - distance
     - energy_liquid
     - time
     - total_cost
   - info
     - cost_aggregation
     - network_state_variable_rates
     - state_variable_coefficients
     - state_variable_indices
     - vehicle_state_variable_rates
 - destination_vertex_uuid
 - iterations
 - origin_vertex_uuid
 - output_plugin_executed_time
 - request
   - destination_name
   - destination_vertex
   - destination_x
   - destination_y
   - model_name
   - origin_name
   - origin_vertex
   - origin_x
   - origin_y
   - query_weight_estimate
   - state_variable_coefficients
     - distance
     - energy_liquid
     - time
 - result_memory_usage_bytes
 - route
   - features
   - type
 - route_edge_count
 - search_app_runtime
 - search_executed_time
 - traversal_summary
   - distance
   - distance_unit
   - energy_liquid
   - time
   - time_unit
   - vehicle_info
     - energy_unit
 - tree_edge_count


### Plotting

We can also plot the results to see the difference between the two routes.

In [72]:
from nrel.routee.compass.plot import plot_route_folium, plot_routes_folium

We can use the `plot_route_folium` function to plot single routes, passing in the `line_kwargs` parameter to customize the folium linestring:

In [73]:
m = plot_route_folium(shortest_time_result, line_kwargs={"color": "blue", "tooltip": "Shortest Time"})
m = plot_route_folium(least_energy_result, line_kwargs={"color": "green", "tooltip": "Least Energy"}, folium_map=m)
m

We can also use the plot_routes_folium function and pass in multiple results. The function will color the routes based on the `value_fn` which takes a single result as an argument. For example, we can tell it to color the routes based on the total energy usage. 

In [75]:
m = plot_routes_folium(results, value_fn=lambda r: r["traversal_summary"]["energy_liquid"], color_map="plasma")
m